# Il giro del mondo in 80 giorni

Consideriamo il dataset che descrive alcune delle principali città del mondo. Supponiamo che sia sempre possibile viaggiare da ciascuna città alle 3 città più vicine e che tale viaggio richieda 2 ore per la città più vicina, 4 ore per la seconda città più vicina e 8 ore per la terza città più vicina. Inoltre, il viaggio dura altre 2 ore se la città di destinazione si trova in un Paese diverso da quello di partenza e altre 2 ore se la città di destinazione ha più di 200.000 abitanti.

Partendo da Londra e viaggiando sempre verso est, è possibile fare il giro del mondo tornando a Londra in 80 giorni? Quanto tempo richiede almeno?

In [1]:
import geopandas as gpd
import geopy.distance
import numpy as np
import pandas as pd
import plotly.express as px
import heapq
import copy

from models import City
from models import prune_cities, dijkstra, normalize_lng

<!-- La mia soluzione e' basata sulla creazione di un grafo di oggetti City a cui si applica l'algoritmo di Dijkstra
per trovare il cammino minimo da Londra a Londra, viaggiando sempre verso est. Eseguo il programma su un dataset ridotto
(ho estratto un sample casuale)


Gli oggetti City:
Ciascun oggetto contiene
- l'anagrafica della citta' (nome, stato, coordinate, popolazione)
- le informazioni che la identificano come nodo nel grafo (la lista di citta' precedenti, ovvero da cui posso raggiungerla;
e la lista delle 3 citta' successive, ovvero quelle che posso raggiungere da qui. Entrambe le liste sono liste di tuple
in quanto contengono anche il numero di ore necessarie allo spostamento)

I metodi della classe sono principalmente orientati alla creazione e al calcolo di questi ultimi campi.


Il grafo:
Si tratta di un grafo pesato, orientato e connesso, rappresentato con liste di adiacenza (le 3 citta' piu' vicine). I nodi
sono oggetti di tipo City
Il peso di ciascun arco corrisponde al numero di ore necessarie per muoversi da un nodo all'altro, calcolate come da consegna -->

In [2]:
# carico il dataset
df = pd.read_excel('samplecities.xlsx')

In [3]:
# Osservo che la longitudine, per come e' definita, non e' molto maneggevole
# Assume infatti valori negli intervalli [0, 180] e [-180, 0], cambiando segno
# quando si passa dall'International Date Line (e visto che la Longitudine e' 0 in corrispondenza
# del meridiano di Greenwich, questo cambio di segno cade proprio in mezzo al nostro percorso.)

# Definisco quindi una funzione che 'trasla' la longitudine in modo da avere una misura
# che prende valori positivi in [0, 360]. Servira' per semplificare il controllo della condizione
# di viaggiare sempre verso est.

# riempo la colonna corrispondente nel dataframe
df['east_of_london'] = df['lng'].apply(normalize_lng)

# inserisco londra_end
df.loc[len(df.index)] = ['London', 51.5072, -0.1275, 'United Kingdom', 'GBR', 'London, City of', 10979000.0, 360.0000]  

# ordino per distanza a est da londra
df = df.sort_values('east_of_london')

# reset degli indici in base al nuovo ordine
df = df.reset_index(drop=True)


In [4]:
df

,city,lat,lng,country,iso3,admin_name,population,east_of_london
0,London,51.5072,-0.1275,United Kingdom,GBR,"London, City of",10979000.0,0.0000
1,Holborn,51.5172,-0.1182,United Kingdom,GBR,Camden,13023.0,0.0093
2,Brixton,51.4630,-0.1060,United Kingdom,GBR,Lambeth,78536.0,0.0215
3,Villareal,39.9378,-0.1014,Spain,ESP,Valencia,50893.0,0.0261
4,Spitalfields,51.5166,-0.0750,United Kingdom,GBR,Tower Hamlets,10286.0,0.0525
...,...,...,...,...,...,...,...,...
1324,Wembley,51.5528,-0.2979,United Kingdom,GBR,Brent,102856.0,359.8296
1325,Acton,51.5135,-0.2707,United Kingdom,GBR,Ealing,62480.0,359.8568
1326,Ewell,51.3500,-0.2490,United Kingdom,GBR,Surrey,34872.0,359.8785
1327,Wandsworth,51.4550,-0.1920,United Kingdom,GBR,Wandsworth,61594.0,359.9355


In [5]:
# creo una lista di oggetti City dalle righe del dataset. Si veda il file models.py per una descrizione completa della classe
cities_list = []

for index, row in df.iterrows():
    cities_list.append(City(
        row["city"],
        row["country"],
        row["iso3"],
        row["population"],
        row["lng"],
        row["lat"],
        row["east_of_london"],
        row["admin_name"],
        index
    )
)

# rinomino London in London_start per distinguerla dalla Londra di arrivo nella rappresentazione degli oggetti (cfr. con implementazione
# di __eq__ nella classe)
cities_list[0].city = 'London_start' 

In [6]:
# creo un dizionario con tutte le distanze in km tra le citta' usando la funzione geopy.distance.great_circle sulle coordinate
dist_dict = {}

for i, p1 in enumerate(zip(df.lat, df.lng)):
    for j, p2 in enumerate(zip(df.lat, df.lng)):
        dist_dict[cities_list[i], cities_list[j]] = geopy.distance.great_circle(p1, p2).km


In [7]:
# per ogni citta', calcola la lista delle 3 citta' piu' vicine ordinate in base alle ore per raggiungerle
for x in cities_list:
    x.succ = x.get_candidates_dict(cities_list, dist_dict)

In [8]:
# calcola le citta' precedenti: se x e' nella lista di successori di un'altra citta' y -> y e' prev di x
for x in cities_list:
    candidates = [y for y in cities_list if y.east_of_london < x.east_of_london]
    for y in candidates: # y e' un tuple (citta', ore di distanza)
        for i in y.succ:
            if x == i[0]:
                x.prev.append((y, i[1]))
            

In [10]:

# fisso il punto di partenza e il punto di arrivo, in questo caso Londra
london_start = cities_list[0]
london_end = cities_list[-1]

# dijkstra: abbiamo un grafo connesso, orientato. per calcolare il cammino minimo da Londra_start a Londra_end uso l'algoritmo di Dijkstra
# implementato con uno heap come coda di priorita' nel file models
(hours, path) = dijkstra(graph, london_start, london_end)

In [11]:
print("total days:",hours / 24)

total days: 13.0


In [12]:
# percorso: 
display(path)

[(London_start, GBR),
 (Spitalfields, GBR),
 (Wanstead, GBR),
 (Tilbury, GBR),
 (Chatham, GBR),
 (Rainham, GBR),
 (Deinze, BEL),
 (Aachen, DEU),
 (Ratingen, DEU),
 (Heiligenhaus, DEU),
 (Lennestadt, DEU),
 (Salzkotten, DEU),
 (Bodenwerder, DEU),
 (Seesen, DEU),
 (Burg, DEU),
 (Wittenberg, DEU),
 (Erkner, DEU),
 (Wolgast, DEU),
 (Kristianstad, SWE),
 (Kungsör, SWE),
 (Kristinestad, FIN),
 (Kauhava, FIN),
 (Kuhmo, FIN),
 (Kola, RUS),
 (Amderma, RUS),
 (Ishim, RUS),
 (Tomsk, RUS),
 (Zheleznogorsk, RUS),
 (Svirsk, RUS),
 (Zabaykalsk, RUS),
 (Gannan, CHN),
 (Xinqing, CHN),
 (Korsakov, RUS),
 (Korf, RUS),
 (Kalifornsky, USA),
 (Mukilteo, USA),
 (Cavalero, USA),
 (Connell, USA),
 (Country Homes, USA),
 (Fernie, CAN),
 (Olds, CAN),
 (Weyburn, CAN),
 (Morden, CAN),
 (Sioux Lookout, CAN),
 (Rothschild, USA),
 (Ashwaubenon, USA),
 (De Pere, USA),
 (North Chicago, USA),
 (Fair Plain, USA),
 (Kendallville, USA),
 (New Haven, USA),
 (Lima, USA),
 (Dublin, USA),
 (Seville, USA),
 (Portage Lakes, USA)

In [13]:
# visualizzazione del percorso su mappa tramite plotly. per un problema di poligoni non riesce a mostrare il tratto
# che attraversa l'International Date Line
# quindi tira una riga che torna all'inizio

import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = [x.longitude for x in path],
    lat = [x.latitude for x in path],
    marker = {'size': 10}))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 10, 'lat': 10},
        'style': "open-street-map",
        'center': {'lon': 170, 'lat': 45},
        'zoom': 1.1})

fig.show()

### conclusioni:
E' sicuramente possibile fare il giro del mondo in 80 giorni se si usa un dataset di dimensioni ridotte. In generale mi aspetto che all'aumentare delle dimensioni del dataset aumenti anche il numero di giorni necessari perche' ci si sposta tra citta' molto piu' ravvicinate. Purtroppo la funzione geopy.distance.great_circle per il calcolo di distanze su una geodetica e' molto lenta, e con il dataset originale avrebbe dovuto fare un numero di conti dell'ordine di (26000)^2, letale per il mio computer
